# Numeric type: DeterminedDigits
Here I am experimenting with a new numeric type in Python, to represent real numbers with so called determined digits.

What makes digits Determined Digits? Well, they live by ten rules of thumb:
1. Digits are fully determined bounds of a number.
2. Digits are not included unless they are correct.
3. Digits are best at representing knowledge.
4. Digits are not suited to representing uncertainty.
5. Digits catch a number by building fences.
6. Digits are not aiming like an arrow at a number.
7. Digits keep the known inside and the unknown outside.
8. Digits always invite new knowledge.
9. Digits can be determined one by one and simply appended.
10. Digits can be removed by truncating, but not by rounding.

Dig this:


In [292]:
import math

Here is an exact number with known digits: 299792.458.

Digits beyond that are false:

In [273]:
import decimal
x=decimal.Decimal(299792.458)
x

Decimal('299792.457999999984167516231536865234375')

How can we avoid false digits? It's easiest to start with defining a new class from scratch:

In [302]:
class DeterminedDigits():

    def __init__(self, is_negative = False, digitstring = "0", exponent = 0):
        self.is_negative = is_negative
        self.digitstring = digitstring
        self.exponent = exponent
        
    def __repr__(self):
        return f"DeterminedDigits(is_negative = {self.is_negative}, digitstring = '{self.digitstring}', exponent = {self.exponent})"
        
    def as_float(self):
        x = int(self.digitstring)
        x *= 10**self.exponent
        if self.is_negative:
            x *= -1
        return x
        
    def append(self, string):
        string = str(string)
        self.digitstring += string
        self.exponent -= len(string)
        
    def truncate(self, number):
        self.digitstring = self.digitstring[:-number]
        self.exponent += number


Let's make our number 299792.458 an instance of DeterminedDigits. It is specified by keyword arguments:

In [303]:
c = DeterminedDigits(digitstring = "299792458", is_negative = False, exponent = -3)

print(c)
print(c.as_float())

DeterminedDigits(is_negative = False, digitstring = '299792458', exponent = -3)
299792.458


It has basic methods for appending digits and removing (truncate):

In [305]:
c.append("123")
print(c.as_float(), c)

299792.458123 DeterminedDigits(is_negative = False, digitstring = '299792458123', exponent = -6)


In [306]:
c.truncate(3)
print(c.as_float(), c)

299792.458 DeterminedDigits(is_negative = False, digitstring = '299792458', exponent = -3)


Let's add an interesting method that can find new digits (if they exist), by doing binary search adapted to decimals. The new method also has a helper method:

In [308]:
class DeterminedDigits():

    def __init__(self, is_negative = False, digitstring = "0", exponent = 0):
        self.is_negative = is_negative
        self.digitstring = digitstring
        self.exponent = exponent
        
    def __repr__(self):
        return f"DeterminedDigits(is_negative = {self.is_negative}, digitstring = '{self.digitstring}', exponent = {self.exponent})"
        
    def as_float(self):
        x = int(self.digitstring)
        x *= 10**self.exponent
        if self.is_negative:
            x *= -1
        return x
        
    def append(self, string):
        string = str(string)
        self.digitstring += string
        self.exponent -= len(string)
        
    def truncate(self, number):
        self.digitstring = self.digitstring[:-number]
        self.exponent += number
        
    # a temporary method for adding a digit:
    def add(self, digit):
        x = int(self.digitstring + str(digit))
        x *= 10**(self.exponent - 1)
        if self.is_negative:
            x *= -1
        return x
                
    def nextdigit(self, binarytest):
        tree = (((0,1,1),2,(2,3,(3,4,4))),5,((5,6,6),7,(7,8,(8,9,9))))
        while (isinstance(tree , tuple)):
            tree = tree[0] if binarytest(self.add(tree[1])) else tree[2]
        return tree
        

## 3.141592653589793238462643383279502884197169399
Let us try to determine the digits of pi. We know that pi is between 0 and 10, so we initialize pi with exponent = 1 (as "zero tens").

In [312]:
pi = DeterminedDigits(exponent = 1) # = zero tens
print(f"pi = {pi}")

def testfunction(x):
    return (math.sin(x) < 0)

for counter in range(10):
    d = pi.nextdigit(testfunction)
    pi.append(d)
    print(pi.as_float())

print(f"Now, pi = {pi}")

pi = DeterminedDigits(is_negative = False, digitstring = '0', exponent = 1)
3
3.1
3.14
3.141
3.1415
3.1415900000000003
3.1415919999999997
3.1415926
3.14159265
3.141592653
Now, pi = DeterminedDigits(is_negative = False, digitstring = '03141592653', exponent = -9)


What I like about DeterminedDigits is that in each step a new digit is found once and for all. This is a tidy convergence! Note that as_float() outputs its own digits, but they are not part of DeterminedDigits.

We can independently resume and determine more digits:

In [313]:
for counter in range(6):
    d = pi.nextdigit(testfunction)
    pi.append(d)
    print(pi.as_float())

print(f"Now, pi = {pi}")
print("\nNot surprisingly, math.pi is:", math.pi)

3.1415926535
3.1415926535799996
3.141592653589
3.1415926535897003
3.14159265358979
3.141592653589793
Now, pi = DeterminedDigits(is_negative = False, digitstring = '03141592653589793', exponent = -15)

Not surprisingly, math.pi is: 3.141592653589793


Likewise, we can determine digits in Euler's number _e_. This time we initially know that _e_ is between 0 and 1000:

In [314]:
e = DeterminedDigits(exponent = 3) # = zero thousands
print(f"e = {e}")

for counter in range(18):
    d = e.nextdigit(lambda x: math.log(x)-1 > 0)
    e.append(d)
    print(e.as_float())
    
print(f"Now, e = {e}")
print("\nNot surprisingly, math.e is:", math.e)

e = DeterminedDigits(is_negative = False, digitstring = '0', exponent = 3)
0
0
2
2.7
2.71
2.718
2.7182
2.71828
2.7182809999999997
2.7182817999999997
2.71828182
2.7182818280000003
2.7182818284
2.71828182845
2.718281828459
2.7182818284590002
2.71828182845904
2.718281828459045
Now, e = DeterminedDigits(is_negative = False, digitstring = '0002718281828459045', exponent = -15)

Not surprisingly, math.e is: 2.718281828459045


Todo:
- https://en.wikipedia.org/wiki/Interval_arithmetic

# Solo Numbers
I define a set of numbers that I call "solo numbers". Solo numbers are a generalization of (the most) round numbers.

## Round numbers, half-round numbers and solo numbers
We celebrate anniversaries that are round (10, 20, 30,...) or half-round (15, 25, 25,...). Half-round numbers are numbers with 5 as the last digit. Round numbers have one or more zeroes at the end, so 110 and 150 are also round, but less round than 100 and 200.

We can define a round number of k'th degree as $k$ non-zero digits followed by one or more zeroes. Second degree round numbers (e.g. 3600 or 270,000) are more round than 3. degree numbers (e.g. 9990 or 495,000) and so on. The most round are the first degree round numbers, e.g. 10, 20, 100, 8000, 50000. How can we extend the set of first degree round numbers below 10?

We can define a generalization of the 1. degree round numbers that include numbers below 10 in the following "logarithmic" way: We include numbers 1-9 and we include 0.1-0.9, etc. A _solo number_ is a number that can be written as $m \times 10^{n}$ where $m$ is a single digit between 1 and 9 and $n$ is an integer (...,-1, 0, 1,...). Examples are 0.0003, 0.9, 1, 5, 200, 7000. They are called "solo" numbers because there is one "man" who does all the work, all other digits are "quiet".

Solo numbers are also defined with other bases than 10. A base-$b$ solo number is $m \times b^{n}$, where $m$ is between 1 and $b-1$. Binary solo numbers have only $m=1$, so they are all $2^{n}$ (equal to 0.25, 0.5, 1, 2, 4, etc.).

It may turn out that the name "solo number" is taken. Other ideas for a name is: One-man number, Rough number, Frontier number, Forward number, Spearhead number, Roundest number, Logscale number, Level number, Major number, First number, Heroic number.

Solo numbers are ordered along the axis, and it is natural to define an _index_, with index _0_ assigned to 1 (since 1 is a solo number for every base):

..., _-10_: 0.09, _-9_: 0.1,..., _-2_: 0.8, _-1_: 0.9, _0_: 1, _1_: 2, _2_: 3,..., _9_: 10, _10_: 20,..., _18_: 100, _19_: 200, _20_: 300,...








## The SoloNumber Project
This project aims at developing a numeric type in Python to represent Solo Numbers and the operations that can be done with them.

The initial plan is to write a class DecimalSoloNumber(), that is derived from a float class, and that internally represents a solo number using three integers: $m$, $b$ and $n$ (in the decimal case $b=10$).

For example, it could be made possible to do things like:

In [6]:
x = DecimalSoloNumber(300)
y = DecimalSoloNumber(0.009)
x.m # is 3
y.n() # is -3
x.nextsolo() # is 400
x.soloindex # is 20
y.soloindex # is -19
z = DecimalSoloNumber(314)
z.nearestsolo() # is 300

NameError: name 'DecimalSoloNumber' is not defined